In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch 
import os
import torchvision
from torchvision import datasets,transforms,models
from torch.utils.data import DataLoader,Dataset
import cv2
from ultralytics import YOLO
import torch.nn as nn
import torch.optim as optim

In [ ]:
os.chdir('/kaggle/input/neu-metal-surface-defects-data/NEU Metal Surface Defects Data/train')

In [ ]:
for file in os.listdir():
    print(file)

In [ ]:
image = cv2.imread('/kaggle/input/neu-metal-surface-defects-data/NEU Metal Surface Defects Data/train/Rolled/RS_126.bmp')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(image)

This should be simple case of image classification

In [ ]:
from torchvision.datasets import ImageFolder

In [ ]:
train_dir = '/kaggle/input/neu-metal-surface-defects-data/NEU Metal Surface Defects Data/train'
valid_dir = '/kaggle/input/neu-metal-surface-defects-data/NEU Metal Surface Defects Data/valid'
test_dir = '/kaggle/input/neu-metal-surface-defects-data/NEU Metal Surface Defects Data/test'

In [ ]:
transformer_func = transforms.Compose([transforms.ToTensor()])

In [ ]:
transformer_func

In [ ]:
train_data = ImageFolder(root = train_dir,transform=transformer_func)
valid_data = ImageFolder(root = valid_dir,transform=transformer_func)

In [ ]:
train_loader = DataLoader(train_data,batch_size=32,num_workers=4,shuffle=True)
valid_loader = DataLoader(valid_data,batch_size=32,num_workers=4,shuffle=True)

In [ ]:
for data,labels in train_loader:
    print(labels.shape)
    print(data.shape)
    break

In [ ]:
train_data.class_to_idx

In [ ]:
#Creating a simple NN
class linearNN(nn.Module):
    def __init__(self):
        super(linearNN,self).__init__()
        # self.flatten = nn.Flatten()
        self.linear_nn = nn.Sequential(
        
            nn.Linear(120000,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,128),
            nn.ReLU(),
            nn.Linear(128,6)
        )
    def forward(self,x):
        x = x.view(x.size(0),-1)
        return self.linear_nn(x)

In [ ]:
class CNNmodel(nn.Module):
    def __init__(self):
        super(CNNmodel,self).__init__()
        self.convlayer = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=32,padding=1,kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32,64,3,1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64,128,3,1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.linearlayer = nn.Sequential(
            # nn.Flatten(),
            nn.Linear(23*23*128,128),
            nn.ReLU(),
            nn.Linear(128,6)
    )
    def forward(self,x):
        x = self.convlayer(x)
        x = x.view(x.size(0),-1)
        x = self.linearlayer(x)
        return x

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = CNNmodel().to(device)

In [ ]:
from torchsummary import summary

summary(model,input_size=(3,200,200))

In [ ]:
#testing if model and images work
for data,labels in train_loader:
    print(labels)
    y = (model(data.to(device)))
    y_pred = torch.argmax(torch.softmax(y,dim=1),dim=1)
    print(y_pred)
    break

In [ ]:
#hyperparams
epochs = 50
lre = 1e-04

In [ ]:
optimizer = optim.Adam(model.parameters(),lr=lre)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
train_losses,test_losses = [],[]
train_accs,test_accs = [],[]
for epoch in range(epochs):
    model.train()
    train_loss,correct = 0,0
    for batch,(x,y) in enumerate(train_loader):
        x,y = x.to(device),y.to(device)
        pred = model(x)
        loss = loss_fn(pred,y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss += loss.item()
        y_pred_class = torch.argmax(torch.softmax(pred,dim=1),dim=1)
        correct += (y_pred_class == y).sum().item()/len(pred)
    avg_loss = train_loss/len(train_loader)
    avg_acc = correct/len(train_loader)
    train_losses.append(avg_loss)
    train_accs.append(avg_acc)
    print(f'Epoch {epoch} | loss: {avg_loss} | accuracy: {avg_acc}')

    with torch.no_grad():
        model.eval()
        test_loss,test_acc = 0,0
        for batch,(x,y) in enumerate(valid_loader):
            x,y = x.to(device),y.to(device)
            pred = model(x)
            loss = loss_fn(pred,y)
            test_loss += loss.item()
            y_pred_class = torch.argmax(torch.softmax(pred,dim=1),dim=1)
            test_acc += (y_pred_class == y).sum().item()/len(pred)
    avg_loss = test_loss/len(valid_loader)
    avg_acc = test_acc/len(valid_loader)
    test_losses.append(avg_loss)
    test_accs.append(avg_acc)
    print(f'Epoch {epoch} | test loss: {avg_loss} | test accuracy: {avg_acc}')
        
        
        

In [ ]:
plt.plot(train_accs)
plt.plot(test_accs)

In [ ]:
plt.plot(train_losses)
plt.plot(test_losses)

RESNET50 Fine tuning

In [ ]:
tl_model = models.resnet50(pretrained=True)

In [ ]:
#This will stop updating weights
for param in tl_model.parameters():
    param.requires_grad = False

In [ ]:
#Modify FCN
num_classes = 6
tl_model.avgpool=nn.AdaptiveAvgPool2d(output_size=(1,1))
input_tolinear = tl_model.fc.in_features
tl_model.fc =nn.Linear(input_tolinear,num_classes)
tl_model.to(device='cuda')

In [ ]:
from torchsummary import summary

summary(tl_model,input_size=(3,224,224))

In [ ]:
transformer_func = transforms.Compose([transforms.ToTensor(),transforms.Resize((224,224))])
train_data = ImageFolder(root = train_dir,transform=transformer_func)
valid_data = ImageFolder(root = valid_dir,transform=transformer_func)
train_loader = DataLoader(train_data,batch_size=32,num_workers=4,shuffle=True)
valid_loader = DataLoader(valid_data,batch_size=32,num_workers=4,shuffle=True)

In [ ]:
tl_optimizer = optim.Adam(tl_model.parameters(),lr=lre)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
epochs= 50

In [ ]:
train_losses,test_losses = [],[]
train_accs,test_accs = [],[]
for epoch in range(epochs):
    tl_model.train()
    train_loss,correct = 0,0
    for batch,(x,y) in enumerate(train_loader):
        x,y = x.to(device),y.to(device)
        pred = tl_model(x)
        loss = loss_fn(pred,y)
        loss.backward()
        tl_optimizer.step()
        tl_optimizer.zero_grad()
        train_loss += loss.item()
        y_pred_class = torch.argmax(torch.softmax(pred,dim=1),dim=1)
        correct += (y_pred_class == y).sum().item()/len(pred)
    avg_loss = train_loss/len(train_loader)
    avg_acc = correct/len(train_loader)
    train_losses.append(avg_loss)
    train_accs.append(avg_acc)
    print(f'Epoch {epoch} | loss: {avg_loss} | accuracy: {avg_acc}')

    with torch.no_grad():
        tl_model.eval()
        test_loss,test_acc = 0,0
        for batch,(x,y) in enumerate(valid_loader):
            x,y = x.to(device),y.to(device)
            pred = tl_model(x)
            loss = loss_fn(pred,y)
            test_loss += loss.item()
            y_pred_class = torch.argmax(torch.softmax(pred,dim=1),dim=1)
            test_acc += (y_pred_class == y).sum().item()/len(pred)
    avg_loss = test_loss/len(valid_loader)
    avg_acc = test_acc/len(valid_loader)
    test_losses.append(avg_loss)
    test_accs.append(avg_acc)
    print(f'Epoch {epoch} | test loss: {avg_loss} | test accuracy: {avg_acc}')
        
        
        

In [ ]:
plt.plot(train_losses)
plt.plot(test_losses)

In [ ]:
plt.plot(train_accs)
plt.plot(test_accs)

In [ ]:
#Test accuracy is 100% Too much to believe